In [135]:
from selenium import webdriver
import pandas as pd
from bs4 import BeautifulSoup
import urllib.request as req
import pandas as pd
import requests
from io import StringIO
import requests
import time
from selenium.webdriver import Chrome
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfparser import PDFParser
import os
from naver_crawler import *
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import time
import re
import parser
import datetime


In [170]:
os.chdir('C:\\Users\\shic\\Desktop\\Crawling\\Desktop\\Project')
delay=3
browser = Chrome()
browser.implicitly_wait(delay)

In [171]:
url="https://kind.krx.co.kr/disclosure/todaydisclosure.do?method=searchTodayDisclosureMain"
browser.get(url)

In [172]:
del df
df = pd.DataFrame({'date':[],'time':[],'mkt_type':[],'stk_code':[],'title':[],'submit':[]})
df

,date,time,mkt_type,stk_code,title,submit


In [178]:
# 조회 페이지 사이즈는 100으로 선택한다.
search_btn=browser.find_element_by_xpath('//*[@id="currentPageSize"]/option[4]')
search_btn.click()
go_btn = browser.find_element_by_xpath('//*[@id="main-contents"]/section[2]/div[2]/a')
go_btn.click()

time.sleep(2)
result = browser.page_source
soup = BeautifulSoup(result, 'html.parser')

# (수동) 페이지 바꾸고 숫자 바꾸기 (0~4)
dd = 4
a=datetime.date.today() - datetime.timedelta(dd)
date = a.strftime('%Y-%m-%d')

total = soup.find_all('tr',{'id':'parkman'})

for i in range(len(total)):
    temp = total[i]
    hour = temp.find_all('td')[0].text

    try:
        mkt_type = temp.find_all('img',{'class':'vmiddle legend'})[0]['alt']
    except:
        mkt_type = ''
    try:
        stk_code = temp.find_all('a',{'id':'companysum'})[0]['title']
    except:
        stk_code = ''
    title = temp.find_all('a',{'href':'#viewer'})[0]['title']
    submit = temp.find_all('td')[3].text
    record  = pd.Series([date,hour,mkt_type,stk_code,title,submit],index = ['date','time','mkt_type','stk_code','title','submit'])
    df = df.append(record, ignore_index = True)
print("1페이지 완료")

for lk in range(2,6):
    
    try:
        next_btn = browser.find_elements_by_class_name('next')
        next_btn[0].click()
        time.sleep(2)
        result = browser.page_source
        soup = BeautifulSoup(result, 'html.parser')
        total = soup.find_all('tr',{'id':'parkman'})

        for i in range(len(total)):
            temp = total[i]
            hour = temp.find_all('td')[0].text

            try:
                mkt_type = temp.find_all('img',{'class':'vmiddle legend'})[0]['alt']
            except:
                mkt_type = ''
            try:
                stk_code = temp.find_all('a',{'id':'companysum'})[0]['title']
            except:
                stk_code = ''
            title = temp.find_all('a',{'href':'#viewer'})[0]['title']
            submit = temp.find_all('td')[3].text
            record  = pd.Series([date,hour,mkt_type,stk_code,title,submit],index = ['date','time','mkt_type','stk_code','title','submit'])
            df = df.append(record, ignore_index = True) 
        print("{}페이지 완료".format(lk))
    except:
        break
# 중복 제거
df = df.drop_duplicates()

1페이지 완료
2페이지 완료
3페이지 완료
4페이지 완료
5페이지 완료


In [179]:
df.shape

(1603, 6)

In [180]:
df

,date,time,mkt_type,stk_code,title,submit
0,2021-07-20,16:32,코스닥,이디티,증권 발행결과(자율공시)(제3자배정 유상증자),이디티
1,2021-07-20,16:32,코스닥,이디티,투자판단 관련 주요경영사항(경영지배인 사임의 건),이디티
2,2021-07-20,16:32,유가증권,교보증권,투자설명서[일괄신고-사채],교보증권
3,2021-07-20,16:31,유가증권,그린케미칼,현금ㆍ현물 배당 결정,그린케미칼
4,2021-07-20,16:29,유가증권,LX홀딩스,"단기과열종목(가격괴리율, 3거래일 단일가매매) 지정 연장(LX홀딩스1우)",유가증권시장본부
...,...,...,...,...,...,...
1603,2021-07-16,09:13,유가증권,현대차증권,계열금융회사의약관에의한금융거래-[CP],현대차증권
1604,2021-07-16,09:01,유가증권,우리금융지주,투자설명서,우리금융지주
1605,2021-07-16,09:00,유가증권,한샘,주식선물 2단계 가격제한폭 확대요건 도달(상승),파생상품시장본부
1606,2021-07-16,08:40,유가증권,NH투자증권,증권발행실적보고서,엔에이치투자증권


In [183]:
df.rename({
    'date':'일자',
    'time':'시간',
    'mkt_type':'시장종류',
    'stk_code':'회사명',
    'title':'공시제목',
    'submit':'제출인'
},axis=1,inplace=True)
df

,일자,시간,시장종류,회사명,공시제목,제출인
0,2021-07-20,16:32,코스닥,이디티,증권 발행결과(자율공시)(제3자배정 유상증자),이디티
1,2021-07-20,16:32,코스닥,이디티,투자판단 관련 주요경영사항(경영지배인 사임의 건),이디티
2,2021-07-20,16:32,유가증권,교보증권,투자설명서[일괄신고-사채],교보증권
3,2021-07-20,16:31,유가증권,그린케미칼,현금ㆍ현물 배당 결정,그린케미칼
4,2021-07-20,16:29,유가증권,LX홀딩스,"단기과열종목(가격괴리율, 3거래일 단일가매매) 지정 연장(LX홀딩스1우)",유가증권시장본부
...,...,...,...,...,...,...
1603,2021-07-16,09:13,유가증권,현대차증권,계열금융회사의약관에의한금융거래-[CP],현대차증권
1604,2021-07-16,09:01,유가증권,우리금융지주,투자설명서,우리금융지주
1605,2021-07-16,09:00,유가증권,한샘,주식선물 2단계 가격제한폭 확대요건 도달(상승),파생상품시장본부
1606,2021-07-16,08:40,유가증권,NH투자증권,증권발행실적보고서,엔에이치투자증권


In [184]:
os.chdir('C:\\Users\\shic\\Desktop\\Crawling\\Desktop\\Project\\1_Data')
df.to_csv('KIND_TODAY_POST_5DAY.csv',index=False)